## Feature Engineering

In [184]:
import datetime, time, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import glob
from pandas_profiling import ProfileReport
import json

from cesium import datasets

from functools import reduce

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

from datetime import datetime, timezone, timedelta

resample_size = "15MIN"
resample_factor = 15

#windowfactor = 15

import warnings

In [185]:
df_merged= pd.read_pickle('data/pickle/df_merged_5.pickle')

In [186]:
df_merged.shape

(17358, 63)

In [187]:
#from sklearn.decomposition import PCA
# Make an instance of the Model


In [188]:
count = np.isinf(df_merged).values.sum()
print("The data frame contains " + str(count) + " infinite values")
print("The Data Frame has",df_merged.isnull().sum().sum(),"missing values.")

The data frame contains 0 infinite values
The Data Frame has 0 missing values.


## Repab Merging Correction

In [189]:
#df_imbalance = pd.read_pickle('data/pickle/df_imbalance.pickle') data/pickle/df_imbalance_de.pickle

Some merging errors appears at the repab price. We will replace it by the imbalance data frame from our prevous notebook.

In [190]:
#df_imbalance

In [191]:
#df_imbalance_temp = df_imbalance["rebap_eur_mwh"].copy()

In [192]:
#df_imbalance_temp.head()

In [193]:
#df_imbalance_temp = pd.DataFrame(data=df_imbalance_temp)

In [194]:
#df_imbalance_temp.tail()

In [195]:
#df_merged.drop(["rebap_eur_mwh"], axis=1, inplace=True)
##df_merged.merge(df_imbalance_temp_2, how='inner')
#df_merged.to_csv("df_merged_pre_drop.csv")
#df_merged= df_merged.merge(df_imbalance_temp, left_index=True, right_index=True)
#df_merged.to_csv("df_merged_post_drop.csv")

In [196]:
#df_merged.shape

## Target as Feature

Create rebap as feature by shifting it for 44 days.

In [197]:
#df_merged.eval('rt_rebap = rebap_eur_mwh.shift(4224)', inplace=True)

In [198]:
df_merged.tail()

,50Hertz_power_mw_x,DE_power_mw_x,DK_power_mw_x,DK1_power_mw_x,TTG_power_mw_x,total_pred_cons,Biomasse[MWh],Wasserkraft[MWh],Wind Offshore[MWh],Wind Onshore[MWh],...,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y,50Hertz_power_mw_y,DE_power_mw_y,DK_power_mw_y,DK1_power_mw_y,TTG_power_mw_y
dt_start_utc,,,,,,,,,,,,,,,,,,,,,
2021-06-30 20:45:00,4194.0,9994.0,756.0,625.0,3352.0,18921.0,1.121,562.0,420.0,1.919,...,468.0,1197.0,740.0,708.0,729.0,2509.0,8001.0,299.0,228.0,3232.0
2021-06-30 21:00:00,4246.0,10153.0,834.0,694.0,3427.0,19354.0,1.123,632.0,431.0,1.874,...,476.0,1212.0,751.0,723.0,736.0,2570.0,8152.0,299.0,228.0,3302.0
2021-06-30 21:15:00,4291.0,10299.0,834.0,694.0,3492.0,19610.0,1.121,595.0,450.0,1.904,...,472.0,1223.0,751.0,723.0,751.0,2595.0,8243.0,299.0,228.0,3364.0
2021-06-30 21:30:00,4319.0,10429.0,834.0,694.0,3555.0,19831.0,1.125,572.0,475.0,1.914,...,469.0,1237.0,751.0,723.0,768.0,2617.0,8330.0,299.0,228.0,3423.0
2021-06-30 21:45:00,4328.0,10546.0,834.0,694.0,3612.0,20014.0,1.128,567.0,450.0,1.908,...,466.0,1244.0,751.0,723.0,778.0,2636.0,8416.0,299.0,228.0,3482.0


In [199]:
correlations = df_merged.corr(method='pearson')
print(correlations['rebap_eur_mwh'].sort_values(ascending=False).to_string())

rebap_eur_mwh                               1.000000
epex_da_de_eur_mwh                          0.243183
generationplusimbalancepriceeurmwhmbadk1    0.147773
generationimbalancepriceeurmwhmbadk1        0.147303
generationplusimbalancepriceeurmwhmbadk2    0.137709
rebap_eur_mwh_PL                            0.134192
priceforconsumptioneurmwhmbadk1             0.130247
generationimbalancepriceeurmwhmbadk2        0.123288
priceforconsumptioneurmwhmbadk2             0.111158
Wasserkraft[MWh]                            0.101527
DE_power_mw_y                               0.080726
Pumpspeicher[MWh]                           0.080279
rebap_eur_mwh_BE                            0.080030
50Hertz_power_mw_y                          0.078830
DK1_power_mw_y                              0.072103
TTG_power_mw_y                              0.063629
Wind Onshore[MWh]                           0.061494
DK_power_mw_y                               0.059533
rebap_eur_mwh_AUS                           0.

In [200]:
df_merged.rename(columns={"Wasserkraft[MWh]": "Wasserkraft", "Braunkohle[MWh]": "Braunkohle"},inplace=True)
df_merged.columns

Index(['50Hertz_power_mw_x', 'DE_power_mw_x', 'DK_power_mw_x',
       'DK1_power_mw_x', 'TTG_power_mw_x', 'total_pred_cons', 'Biomasse[MWh]',
       'Wasserkraft', 'Wind Offshore[MWh]', 'Wind Onshore[MWh]',
       'Photovoltaik[MWh]', 'Sonstige Erneuerbare[MWh]', 'Kernenergie[MWh]',
       'Braunkohle', 'Steinkohle[MWh]', 'Erdgas[MWh]', 'Pumpspeicher[MWh]',
       'Sonstige Konventionelle[MWh]', 'rel_total', 'diff_prog_real',
       'rebap_eur_mwh', 'rz_saldo_mwh', 'rebap_eur_mwh_BE', 'rz_saldo_mwh_BE',
       'rebap_eur_mwh_PL', 'rz_saldo_mwh_PL', 'rebap_eur_mwh_AUS',
       'rz_saldo_mwh_AUS', 'generationplusimbalancepriceeurmwhmbadk2',
       'generationimbalancepriceeurmwhmbadk2',
       'priceforconsumptioneurmwhmbadk2', 'totalimbalancemwhmbadk2',
       'generationplusimbalancepriceeurmwhmbadk1',
       'generationimbalancepriceeurmwhmbadk1',
       'priceforconsumptioneurmwhmbadk1', 'totalimbalancemwhmbadk1',
       'sechs_h_regelung', 'epex_da_de_eur_mwh', '50Hertz_power_mw_x',

In [201]:
#df_merged["Braunkohle_diff"] = df_merged["Braunkohle"]-df_merged["Braunkohle"].shift()

#df_merged["roc_rz_saldo_mwh"] = (df_merged["rz_saldo_mwh"] - df_merged["rz_saldo_mwh"].shift())/df_merged["rz_saldo_mwh"].shift().abs()

#df_merged.eval('Braunkohle_roll_win16 = Braunkohle.rolling(window=60).mean()')
#df_merged.eval('Braunkohle_roll_win8 = Braunkohle.rolling(window=30).mean()')
#df_merged.eval('Braunkohle_roll_win4 = Braunkohle.rolling(window=15).mean()')
#df_merged.eval('Braunkohle_roll_win2 = Braunkohle.rolling(window=7).mean()')

df_merged.eval('Braunkohle_roll_diff_32_8 = Braunkohle.rolling(window=16).mean() - Braunkohle.rolling(window=4).mean()', inplace = True)

In [202]:


#df_merged["Wasserkraft_diff"] = df_merged["Wasserkraft"]-df_merged["Wasserkraft"].shift()

#df_merged["roc_rz_saldo_mwh"] = (df_merged["rz_saldo_mwh"] - df_merged["rz_saldo_mwh"].shift())/df_merged["rz_saldo_mwh"].shift().abs()
df_merged.eval('Wasserkraft_roll_diff_16_2 = Wasserkraft.rolling(window=4).mean() - Wasserkraft.rolling(window=2).mean()', inplace = True)


In [203]:
df_merged["rz_saldo_mwh_diff"] = df_merged["rz_saldo_mwh"]-df_merged["rz_saldo_mwh"].shift()

#df_merged["roc_rz_saldo_mwh"] = (df_merged["rz_saldo_mwh"] - df_merged["rz_saldo_mwh"].shift())/df_merged["rz_saldo_mwh"].shift().abs()

df_merged.eval('rz_saldo_roll_win16 = rz_saldo_mwh.rolling(window=16).mean()', inplace = True)
df_merged.eval('rz_saldo_roll_win4 = rz_saldo_mwh.rolling(window=4).mean()', inplace = True)
df_merged.eval('rz_saldo_roll_win2 = rz_saldo_mwh.rolling(window=2).mean()', inplace = True)

df_merged.eval('rz_saldo_roll_diff_16_2 = rz_saldo_roll_win16 - rz_saldo_roll_win2', inplace = True)


In [204]:
df_merged["epex_diff"] = df_merged["epex_da_de_eur_mwh"]-df_merged["epex_da_de_eur_mwh"].shift()

#df_merged["roc_epex"] = (df_merged["epex_da_de_eur_mwh"] - df_merged["epex_da_de_eur_mwh"].shift())/df_merged["epex_da_de_eur_mwh"].shift().abs()

df_merged.eval('epex_roll_win8 = epex_da_de_eur_mwh.rolling(window=8).mean()', inplace = True)
#df_merged.eval('epex_roll_win4 = epex_da_de_eur_mwh.rolling(window=4).mean()', inplace = True)
df_merged.eval('epex_roll_win2 = epex_da_de_eur_mwh.rolling(window=4).mean()', inplace = True)

df_merged.eval('epex_roll_diff_16_2 = epex_da_de_eur_mwh.rolling(window=16).mean() - epex_roll_win2', inplace = True)

In [205]:
#df_merged.eval('dk1_roll_win16 = generation_/_+_imbalance_price__eur/mwh__-_mba|dk1.rolling(window=240).mean()', inplace = True)
#df_merged.eval('dk1_roll_win8 = generation_/_+_imbalance_price__eur/mwh__-_mba|dk1.rolling(window=120).mean()', inplace = True)
#df_merged.eval('dk1_roll_win4 = generation_/_+_imbalance_price__eur/mwh__-_mba|dk1.rolling(window=60).mean()', inplace = True)
#df_merged.eval('dk1_roll_win2 = generation_/_+_imbalance_price__eur/mwh__-_mba|dk1.rolling(window=30).mean()', inplace = True)

#df_merged.eval('dk1_saldo_mwh_roll_diff_16_2 = dk1_roll_win16 - dk1_roll_win2', inplace = True)
#df_merged.eval('dk1_saldo_mwh_roll_diff_8_4 = dk1_roll_win8 - dk1_roll_win4', inplace = True)

In [206]:
#df_merged.eval('DIR_ImbalancePrice_PLN_MWh_MBA_PL = ImbalancePrice_PLN_MWh_MBA_PL / ImbalancePrice_PLN_MWh_MBA_PL.abs()', inplace = True)

In [207]:
df_merged.eval('fac_epex_rz_saldo = epex_da_de_eur_mwh * rz_saldo_mwh', inplace = True)

In [208]:
#df_merged.fillna(0, inplace=True)
df_merged.sort_index();

In [209]:
df_merged.to_csv("df_test.csv")

In [210]:
df_merged.shape

(17358, 75)

In [211]:
correlations = df_merged.corr(method='pearson')
print(correlations['rebap_eur_mwh'].sort_values(ascending=False).to_string())

rebap_eur_mwh                               1.000000
rz_saldo_roll_diff_16_2                     0.249470
epex_roll_win2                              0.247754
epex_roll_win8                              0.244680
epex_da_de_eur_mwh                          0.243183
generationplusimbalancepriceeurmwhmbadk1    0.147773
generationimbalancepriceeurmwhmbadk1        0.147303
generationplusimbalancepriceeurmwhmbadk2    0.137709
rebap_eur_mwh_PL                            0.134192
priceforconsumptioneurmwhmbadk1             0.130247
generationimbalancepriceeurmwhmbadk2        0.123288
priceforconsumptioneurmwhmbadk2             0.111158
Wasserkraft                                 0.101527
DE_power_mw_y                               0.080726
Pumpspeicher[MWh]                           0.080279
rebap_eur_mwh_BE                            0.080030
50Hertz_power_mw_y                          0.078830
DK1_power_mw_y                              0.072103
TTG_power_mw_y                              0.

In [212]:
df_merged.shape

(17358, 75)

In [213]:
df_merged.astype(np.float32)

,50Hertz_power_mw_x,DE_power_mw_x,DK_power_mw_x,DK1_power_mw_x,TTG_power_mw_x,total_pred_cons,Biomasse[MWh],Wasserkraft,Wind Offshore[MWh],Wind Onshore[MWh],...,rz_saldo_mwh_diff,rz_saldo_roll_win16,rz_saldo_roll_win4,rz_saldo_roll_win2,rz_saldo_roll_diff_16_2,epex_diff,epex_roll_win8,epex_roll_win2,epex_roll_diff_16_2,fac_epex_rz_saldo
dt_start_utc,,,,,,,,,,,,,,,,,,,,,
2021-01-01 02:30:00,1051.0,2590.0,131.0,54.0,854.0,4680.0,1.138,295.0,95.0,890.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2160.939697
2021-01-01 02:45:00,1027.0,2536.0,131.0,54.0,827.0,4575.0,1.139,295.0,90.0,860.000,...,22.0,NaN,NaN,63.0,NaN,-0.699121,NaN,NaN,NaN,3023.448242
2021-01-01 03:00:00,1004.0,2486.0,105.0,43.0,801.0,4439.0,1.138,294.0,80.0,843.000,...,-18.0,NaN,NaN,65.0,NaN,-0.467409,NaN,NaN,NaN,2261.840088
2021-01-01 03:15:00,989.0,2452.0,105.0,43.0,783.0,4372.0,1.137,290.0,76.0,833.000,...,14.0,NaN,63.00,63.0,NaN,-0.235698,NaN,40.739559,NaN,2810.801270
2021-01-01 03:30:00,978.0,2423.0,105.0,43.0,765.0,4314.0,1.139,292.0,73.0,812.000,...,-151.0,NaN,29.75,-5.5,NaN,-0.003986,NaN,40.388008,NaN,-3252.175537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-30 20:45:00,4194.0,9994.0,756.0,625.0,3352.0,18921.0,1.121,562.0,420.0,1.919,...,136.0,-108.0000,-131.25,-147.0,39.0000,-3.462849,94.427879,91.704185,4.958381,-6859.005371
2021-06-30 21:00:00,4246.0,10153.0,834.0,694.0,3427.0,19354.0,1.123,632.0,431.0,1.874,...,-12.0,-111.6250,-142.25,-85.0,-26.6250,-2.462854,92.845375,88.791687,6.895256,-7676.759766
2021-06-30 21:15:00,4291.0,10299.0,834.0,694.0,3492.0,19610.0,1.121,595.0,450.0,1.904,...,34.0,-116.3750,-110.50,-74.0,-42.3750,-1.462860,91.078316,86.091423,8.542856,-4725.137207


In [214]:
count = np.isinf(df_merged).values.sum()
print("The data frame contains " + str(count) + " infinite values")
print("The Data Frame has",df_merged.isnull().sum().sum(),"missing values.")

The data frame contains 0 infinite values
The Data Frame has 79 missing values.


In [215]:
print(df_merged.index[np.isinf(df_merged).any(1)])
print(df_merged.columns.to_series()[np.isinf(df_merged).any()])

DatetimeIndex([], dtype='datetime64[ns]', name='dt_start_utc', freq=None)
Series([], dtype: object)


In [216]:
#inf_plus = df_merged.index[np.isposinf(df_merged).any(1)].tolist()
#print(len(inf_plus))
#inf_minus = df_merged.index[np.isneginf(df_merged).any(1)].tolist()
#print(len(inf_minus))
#inf_rows = inf_plus + inf_minus

#print(inf_rows)

#max_ = df_merged.loc[df_merged['roc_epex'] != np.inf, 'roc_epex'].max()
#min_ = df_merged.loc[df_merged['roc_epex'] != -np.inf, 'roc_epex'].min()

#for row in inf_rows:
#    if np.isposinf(df_merged.loc[row]["roc_epex"]):
#        df_merged.loc[row]["roc_epex"] = max_
#    elif np.isneginf(df_merged.loc[row]["roc_epex"]):
#        df_merged.loc[row]["roc_epex"] = min_
#    else:
#        pass

In [217]:
count = np.isinf(df_merged).values.sum()
print("The data frame contains " + str(count) + " infinite values")
print("The Data Frame has",df_merged.isnull().sum().sum(),"missing values.")

The data frame contains 0 infinite values
The Data Frame has 79 missing values.


## Data Extension

In [218]:
import datetime

df_merged['weekday'] = df_merged.index.weekday
#df_merged['hour'] = df_merged.index.hour

df_merged['businessday'] = np.where((df_merged.index.weekday) < 5,1,0)

In [219]:
df_merged.shape

(17358, 77)

In [220]:
count = np.isinf(df_merged).values.sum()
print("The data frame contains " + str(count) + " infinite values")
print("The Data Frame has",df_merged.isnull().sum().sum(),"missing values.")

The data frame contains 0 infinite values
The Data Frame has 79 missing values.


In [221]:
df_merged.to_csv("df_test.csv")

Decide between business hours

In [222]:
m1 = df_merged.index.dayofweek >= 0
m2 = df_merged.index.dayofweek < 5
m3 = df_merged.index.hour >= 5 
m4 = df_merged.index.hour <= 18

df_merged["businesshours"] = int(-1)
df_merged['businesshours'] = df_merged['businesshours'].mask(m1 & m2 & m3 & m4, '1')
df_merged["businesshours"] = df_merged["businesshours"].astype('int64')

## Extract mathematical sign 

In [223]:
#df_merged.dropna(axis=0,inplace=True)

In [224]:
df_merged.eval('rz_saldo_mwh_sign = rz_saldo_mwh/abs(rz_saldo_mwh)', inplace=True)

In [225]:
#df_merged.head()

In [226]:
#df_merged.dropna(axis=0,inplace=True)

In [227]:
df_merged.shape

(17358, 79)

In [228]:
correlations = df_merged.corr(method='pearson')
print(correlations['rebap_eur_mwh'].sort_values(ascending=False).to_string())

rebap_eur_mwh                               1.000000
rz_saldo_roll_diff_16_2                     0.249470
epex_roll_win2                              0.247754
epex_roll_win8                              0.244680
epex_da_de_eur_mwh                          0.243183
generationplusimbalancepriceeurmwhmbadk1    0.147773
generationimbalancepriceeurmwhmbadk1        0.147303
generationplusimbalancepriceeurmwhmbadk2    0.137709
rebap_eur_mwh_PL                            0.134192
priceforconsumptioneurmwhmbadk1             0.130247
generationimbalancepriceeurmwhmbadk2        0.123288
priceforconsumptioneurmwhmbadk2             0.111158
Wasserkraft                                 0.101527
businessday                                 0.096346
businesshours                               0.091715
DE_power_mw_y                               0.080726
Pumpspeicher[MWh]                           0.080279
rebap_eur_mwh_BE                            0.080030
50Hertz_power_mw_y                          0.

In [229]:
df_merged.drop(["Biomasse[MWh]","totalimbalancemwhmbadk2", 
                "Photovoltaik[MWh]", "Wind Offshore[MWh]",
               "totalimbalancemwhmbadk1"],axis=1,inplace=True)

In [230]:
#cols = df_merged.columns.tolist()

#for col in cols:
#    print(col)
#    if np.abs(df_merged["rebap_eur_mwh"].corr(df_merged[col])) < 0.05:
#        df_merged.drop([col], axis=1, inplace=True)
#    else:
#        pass        

In [231]:
#np.abs(df_merged["rebap_eur_mwh"].corr(df_merged["rz_saldo_mwh"]))

In [232]:
#df_merged = df_merged.loc['2021-01-01 00:00:00':'2021-07-06 22:45:00']

In [233]:
df_merged.shape

(17358, 74)

In [234]:
df_merged.dropna(axis=0,inplace=True)

In [235]:
df_merged.head()

,50Hertz_power_mw_x,DE_power_mw_x,DK_power_mw_x,DK1_power_mw_x,TTG_power_mw_x,total_pred_cons,Wasserkraft,Wind Onshore[MWh],Sonstige Erneuerbare[MWh],Kernenergie[MWh],...,rz_saldo_roll_diff_16_2,epex_diff,epex_roll_win8,epex_roll_win2,epex_roll_diff_16_2,fac_epex_rz_saldo,weekday,businessday,businesshours,rz_saldo_mwh_sign
dt_start_utc,,,,,,,,,,,,,,,,,,,,,
2021-01-01 06:15:00,915.0,2337.0,81.0,41.0,730.0,4104.0,287.0,552.0,56.0,2.036,...,21.2500,0.163778,39.865196,40.007385,0.151559,-966.090674,4,1,1,-1.0
2021-01-01 06:30:00,910.0,2342.0,81.0,41.0,738.0,4112.0,286.0,536.0,56.0,2.036,...,48.5000,0.163052,39.929604,40.171526,-0.083814,-3233.346434,4,1,1,-1.0
2021-01-01 06:45:00,906.0,2352.0,81.0,41.0,748.0,4128.0,286.0,505.0,57.0,2.039,...,20.5000,0.294637,40.054093,40.368019,-0.289428,1058.498157,4,1,1,1.0
2021-01-01 07:00:00,903.0,2363.0,84.0,52.0,759.0,4161.0,287.0,514.0,57.0,2.039,...,-46.1875,0.558532,40.259093,40.663019,-0.529428,2187.310000,4,1,1,1.0
2021-01-01 07:15:00,882.0,2361.0,84.0,52.0,777.0,4156.0,290.0,510.0,57.0,2.039,...,-69.5000,0.822428,40.565033,41.122681,-0.867957,3072.747225,4,1,1,1.0


In [236]:
df_merged.dropna(axis=0,inplace=True)

In [237]:
df_merged.shape

(17292, 74)

## Save Data

In [238]:
df_merged.to_csv("df_merged_15min.csv")

In [239]:
df_merged

,50Hertz_power_mw_x,DE_power_mw_x,DK_power_mw_x,DK1_power_mw_x,TTG_power_mw_x,total_pred_cons,Wasserkraft,Wind Onshore[MWh],Sonstige Erneuerbare[MWh],Kernenergie[MWh],...,rz_saldo_roll_diff_16_2,epex_diff,epex_roll_win8,epex_roll_win2,epex_roll_diff_16_2,fac_epex_rz_saldo,weekday,businessday,businesshours,rz_saldo_mwh_sign
dt_start_utc,,,,,,,,,,,,,,,,,,,,,
2021-01-01 06:15:00,915.0,2337.0,81.0,41.0,730.0,4104.0,287.0,552.000,56.0,2.036,...,21.2500,0.163778,39.865196,40.007385,0.151559,-966.090674,4,1,1,-1.0
2021-01-01 06:30:00,910.0,2342.0,81.0,41.0,738.0,4112.0,286.0,536.000,56.0,2.036,...,48.5000,0.163052,39.929604,40.171526,-0.083814,-3233.346434,4,1,1,-1.0
2021-01-01 06:45:00,906.0,2352.0,81.0,41.0,748.0,4128.0,286.0,505.000,57.0,2.039,...,20.5000,0.294637,40.054093,40.368019,-0.289428,1058.498157,4,1,1,1.0
2021-01-01 07:00:00,903.0,2363.0,84.0,52.0,759.0,4161.0,287.0,514.000,57.0,2.039,...,-46.1875,0.558532,40.259093,40.663019,-0.529428,2187.310000,4,1,1,1.0
2021-01-01 07:15:00,882.0,2361.0,84.0,52.0,777.0,4156.0,290.0,510.000,57.0,2.039,...,-69.5000,0.822428,40.565033,41.122681,-0.867957,3072.747225,4,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-30 20:45:00,4194.0,9994.0,756.0,625.0,3352.0,18921.0,562.0,1.919,36.0,1.656,...,39.0000,-3.462849,94.427878,91.704189,4.958381,-6859.005479,2,1,-1,-1.0
2021-06-30 21:00:00,4246.0,10153.0,834.0,694.0,3427.0,19354.0,632.0,1.874,36.0,1.657,...,-26.6250,-2.462854,92.845378,88.791689,6.895256,-7676.760000,2,1,-1,-1.0
2021-06-30 21:15:00,4291.0,10299.0,834.0,694.0,3492.0,19610.0,595.0,1.904,36.0,1.657,...,-42.3750,-1.462860,91.078313,86.091424,8.542857,-4725.137000,2,1,-1,-1.0


In [240]:
df_merged.shape

(17292, 74)

In [241]:
df_merged.to_pickle('data/pickle/df_merged_6.pickle')

In [242]:
df_merged.shape

(17292, 74)

In [243]:
count = np.isinf(df_merged).values.sum()
print("The data frame contains " + str(count) + " infinite values")
print("The Data Frame has",df_merged.isnull().sum().sum(),"missing values.")

The data frame contains 0 infinite values
The Data Frame has 0 missing values.


In [254]:
correlations = df_merged.corr(method='pearson')
#print(correlations['rebap_eur_mwh'].sort_values(ascending=False).to_string())
print(correlations['rebap_eur_mwh'].abs().sort_values(ascending=False).to_string())

rebap_eur_mwh                               1.000000
rz_saldo_mwh                                0.367141
rz_saldo_roll_win2                          0.350690
fac_epex_rz_saldo                           0.348333
rz_saldo_roll_win4                          0.320860
rz_saldo_roll_diff_16_2                     0.249584
epex_roll_win2                              0.247906
epex_roll_win8                              0.244915
epex_da_de_eur_mwh                          0.243269
rz_saldo_mwh_sign                           0.241534
rz_saldo_roll_win16                         0.214535
generationplusimbalancepriceeurmwhmbadk1    0.147934
generationimbalancepriceeurmwhmbadk1        0.147409
generationplusimbalancepriceeurmwhmbadk2    0.137813
rebap_eur_mwh_PL                            0.134344
priceforconsumptioneurmwhmbadk1             0.130284
generationimbalancepriceeurmwhmbadk2        0.123284
priceforconsumptioneurmwhmbadk2             0.111107
rz_saldo_mwh_diff                           0.